In [1]:
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
import json
import pickle
import string

In [ ]:
class DiacriticDataset(Dataset):
    def __init__(self,dataset_path,letter_to_id_path,id_to_letter_path,diacritic_to_id_path,id_to_diacritic_path,word_to_id_path,id_to_word_path):
        
        self.file = dataset_path
        
        letter_to_id_file= open(letter_to_id_path, 'rb')
        self.letter_to_id = pickle.load(letter_to_id_file)
        letter_to_id_file.close()
        
        id_to_letter_file = open(id_to_letter_path, 'rb')
        self.id_to_letter = pickle.load(id_to_letter_file)
        id_to_letter_file.close()
        
        diacritic_to_id_file= open(diacritic_to_id_path, 'rb')
        self.diacritic_to_id = pickle.load(diacritic_to_id_file)
        diacritic_to_id_file.close()
        
        id_to_diacritic_file = open(id_to_diacritic_path, 'rb')
        self.id_to_diacritic = pickle.load(id_to_diacritic_file)
        id_to_diacritic_file.close()
        
        word_to_id_file= open(word_to_id_path, 'rb')
        self.word_to_id = pickle.load(word_to_id_file)
        word_to_id_file.close()
        
        id_to_word_file = open(id_to_word_path, 'rb')
        self.id_to_word = pickle.load(id_to_word_file)
        id_to_word_file.close()
        
        self.data = self.prepare_dataset()
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        return self.data[index]
    
    def prepare_dataset():
        

In [2]:
file = open('Dataset/letter_to_id.pickle','rb')
data = pickle.load(file)
file.close

file = open('Dataset/id.pickle','rb')
data = pickle.load(file)
file.close

<function BufferedReader.close>

In [3]:
data

{'و': 0,
 'ل': 1,
 ' ': 2,
 'ج': 3,
 'م': 4,
 'ع': 5,
 'ث': 6,
 'ت': 7,
 'ر': 8,
 'ك': 9,
 'ن': 10,
 'ا': 11,
 'أ': 12,
 'ى': 13,
 'ب': 14,
 'ط': 15,
 'ي': 16,
 'د': 17,
 'ه': 18,
 'ة': 19,
 'ف': 20,
 'إ': 21,
 'ق': 22,
 'ز': 23,
 'ؤ': 24,
 'ض': 25,
 'ذ': 26,
 'س': 27,
 'خ': 28,
 'ح': 29,
 'ظ': 30,
 'ء': 31,
 'ش': 32,
 'غ': 33,
 'ص': 34,
 'ئ': 35,
 'آ': 36}